Universidade Federal do Rio Grande  
Instituto de Oceanografia  
Programa de Pós-graduação em Oceanologia  
**Disciplina**: Análises de Séries Temporais em Oceanografia – 2023  
**Estudante**: Andrés Eloy Piñango Jauregui (153423)  

# Lista de exercícios 6: Wavelets  
****
**Atividade**:  
Escolha um conjunto de dados na forma de uma série temporal (utilize os dados disponibilizados SST.mat ou os seus próprios dados) de tamanho arbitrário para realizar um processo de filtragem de ciclos de variabilidade e análise de espectro de energia. O programa deve ser montado de forma que possa ser realizada a análise do espectro de energia com e sem a utilização de filtro. A análise de espectro de energia pode ser realizada pela utilização da transformada de Fourier (ou função pwelch.m). Deve-se, ainda, utilizar 3 tipos diferentes de Wavelet (utilize os exemplos ASTO_Wavelets_exemplos1.m e ASTO_Wavelets_exemplos2.m como base para a explicação e entrega dos resultados) para comparação dos resultados, bem como analisar com e sem a normalização dos dados.  
  
Os dados utilizados devem estar organizados, já terem passado por um pré-tratamento e possíveis interpolações, caso necessário. O programa deve trazer uma sucinta descrição (ao longo do próprio código) explicando cada procedimento de tratamento adotado e as justificativas para as escolhas dos mesmos.  
  
Uma pequena discussão deve ser inserida ao final do código de forma a justificar os resultados obtidos pela análise de espectro de energiua com e sem a utilização do filtro. Bem como as diferenças relacionadas ao uso de diferentes Wavelets.


****
### Parte 1: Carrega las bibliotecas e funções a ser usadas
A função para fazer a transformada Wavelet foi desenvolvida originalmente por Christopher Torrence & Gilbert P.Compo e mais imformação pode ser encontrada [no paper de 1998](https://doi.org/10.1175/1520-0477(1998)079<0061:APGTWA>2.0.CO;2) ou no [site web](https://paos.colorado.edu/research/wavelets/). A versão para Python foi escrita pela [Evgeniya Predybaylo](mailto:predybaylo.evgenia@gmail.com) em 2014 e editada por Michael von Papen em 2018 para incluir análises em frequências arbitrárias. O codigo pode ser baixado do [Github](https://github.com/ct6502/wavelets).

In [ ]:
### ------------------------------------------------------------------------------------
### Load the libraries to be used in the analysis
### ------------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.optimize import fminbound
from scipy.special._ufuncs import gamma, gammainc

In [ ]:
### ------------------------------------------------------------------------------------
### Define a function to detrend the data
### ------------------------------------------------------------------------------------
def cubic_detrend(series):
    new_series = series.reset_index(drop=True)
    clean_series = new_series.dropna()
    regression = np.polynomial.Polynomial.fit(
        clean_series.index.values, clean_series.values, deg=3
    )
    coef = regression.convert().coef
    detrended_series = series - (
        (coef[3] * new_series.index.values**3)
        + (coef[2] * new_series.index.values**2)
        + (coef[1] * new_series.index.values)
        + coef[0]
    )
    return detrended_series

In [ ]:
### ------------------------------------------------------------------------------------
### Internal function used by CHISQUARE_INV
### Copyright (C) 1995-2021, Christopher Torrence and Gilbert P.Compo
### Python version written by Evgeniya Predybaylo (2014) and edited by Michael von Papen (2018)
### ------------------------------------------------------------------------------------
def chisquare_solve(XGUESS, P, V):
    PGUESS = gammainc(V / 2, V * XGUESS / 2)  # incomplete Gamma function
    PDIFF = np.abs(PGUESS - P)  # error in calculated P
    TOL = 1e-4
    if PGUESS >= 1 - TOL:  # if P is very close to 1 (i.e. a bad guess)
        PDIFF = XGUESS  # then just assign some big number like XGUESS
    return PDIFF

In [ ]:
### ------------------------------------------------------------------------------------
### Inverse of chi-square cumulative distribution function (cdf)
### Copyright (C) 1995-2021, Christopher Torrence and Gilbert P.Compo
### Python version written by Evgeniya Predybaylo (2014) and edited by Michael von Papen (2018)
### ------------------------------------------------------------------------------------
def chisquare_inv(P, V):
    if (1 - P) < 1e-4:
        print("P must be < 0.9999")
    if P == 0.95 and V == 2:  # this is a no-brainer
        X = 5.9915
        return X
    MINN = 0.01  # hopefully this is small enough
    MAXX = 1  # actually starts at 10 (see while loop below)
    X = 1
    TOLERANCE = 1e-4  # this should be accurate enough
    while (X + TOLERANCE) >= MAXX:  # should only need to loop thru once
        MAXX = MAXX * 10.0
        # this calculates value for X, NORMALIZED by V
        X = fminbound(chisquare_solve, MINN, MAXX, args=(P, V), xtol=TOLERANCE)
        MINN = MAXX
    X = X * V  # put back in the goofy V factor
    return X

In [ ]:
### ------------------------------------------------------------------------------------
### Significance testing for the 1D Wavelet transform WAVELET
### Copyright (C) 1995-2021, Christopher Torrence and Gilbert P.Compo
### Python version written by Evgeniya Predybaylo (2014) and edited by Michael von Papen (2018)
### ------------------------------------------------------------------------------------
def wave_signif(
    Y,
    dt,
    scale,
    sigtest=0,
    lag1=0.0,
    siglvl=0.95,
    dof=None,
    mother="MORLET",
    param=None,
    gws=None,
):
    n1 = len(np.atleast_1d(Y))
    J1 = len(scale) - 1
    dj = np.log2(scale[1] / scale[0])

    if n1 == 1:
        variance = Y
    else:
        variance = np.std(Y) ** 2

    # get the appropriate parameters [see Table(2)]
    if mother == "MORLET":  # ----------------------------------  Morlet
        empir = [2.0, -1, -1, -1]
        if param is None:
            param = 6.0
            empir[1:] = [0.776, 2.32, 0.60]
        k0 = param
        # Scale-->Fourier [Sec.3h]
        fourier_factor = (4 * np.pi) / (k0 + np.sqrt(2 + k0**2))
    elif mother == "PAUL":
        empir = [2, -1, -1, -1]
        if param is None:
            param = 4
            empir[1:] = [1.132, 1.17, 1.5]
        m = param
        fourier_factor = (4 * np.pi) / (2 * m + 1)
    elif mother == "DOG":  # -------------------------------------Paul
        empir = [1.0, -1, -1, -1]
        if param is None:
            param = 2.0
            empir[1:] = [3.541, 1.43, 1.4]
        elif param == 6:  # --------------------------------------DOG
            empir[1:] = [1.966, 1.37, 0.97]
        m = param
        fourier_factor = 2 * np.pi * np.sqrt(2.0 / (2 * m + 1))
    else:
        print("Mother must be one of MORLET, PAUL, DOG")

    period = scale * fourier_factor
    dofmin = empir[0]  # Degrees of freedom with no smoothing
    Cdelta = empir[1]  # reconstruction factor
    gamma_fac = empir[2]  # time-decorrelation factor
    dj0 = empir[3]  # scale-decorrelation factor

    freq = dt / period  # normalized frequency

    if gws is not None:  # use global-wavelet as background spectrum
        fft_theor = gws
    else:
        # [Eqn(16)]
        fft_theor = (1 - lag1**2) / (
            1 - 2 * lag1 * np.cos(freq * 2 * np.pi) + lag1**2
        )
        fft_theor = variance * fft_theor  # include time-series variance

    signif = fft_theor
    if dof is None:
        dof = dofmin

    if sigtest == 0:  # no smoothing, DOF=dofmin [Sec.4]
        dof = dofmin
        chisquare = chisquare_inv(siglvl, dof) / dof
        signif = fft_theor * chisquare  # [Eqn(18)]
    elif sigtest == 1:  # time-averaged significance
        if len(np.atleast_1d(dof)) == 1:
            dof = np.zeros(J1) + dof
        dof[dof < 1] = 1
        # [Eqn(23)]
        dof = dofmin * np.sqrt(1 + (dof * dt / gamma_fac / scale) ** 2)
        dof[dof < dofmin] = dofmin  # minimum DOF is dofmin
        for a1 in range(0, J1 + 1):
            chisquare = chisquare_inv(siglvl, dof[a1]) / dof[a1]
            signif[a1] = fft_theor[a1] * chisquare
    elif sigtest == 2:  # time-averaged significance
        if len(dof) != 2:
            print("ERROR: DOF must be set to [S1,S2]," " the range of scale-averages")
        if Cdelta == -1:
            print(
                "ERROR: Cdelta & dj0 not defined"
                " for " + mother + " with param = " + str(param)
            )

        s1 = dof[0]
        s2 = dof[1]
        avg = np.logical_and(scale >= 2, scale < 8)  # scales between S1 & S2
        navg = np.sum(np.array(np.logical_and(scale >= 2, scale < 8), dtype=int))
        if navg == 0:
            print("ERROR: No valid scales between " + s1 + " and " + s2)
        Savg = 1.0 / np.sum(1.0 / scale[avg])  # [Eqn(25)]
        Smid = np.exp((np.log(s1) + np.log(s2)) / 2.0)  # power-of-two midpoint
        dof = (dofmin * navg * Savg / Smid) * np.sqrt(
            1 + (navg * dj / dj0) ** 2
        )  # [Eqn(28)]
        fft_theor = Savg * np.sum(fft_theor[avg] / scale[avg])  # [Eqn(27)]
        chisquare = chisquare_inv(siglvl, dof) / dof
        signif = (dj * dt / Cdelta / Savg) * fft_theor * chisquare  # [Eqn(26)]
    else:
        print("ERROR: sigtest must be either 0, 1, or 2")

    return signif

In [ ]:
### ------------------------------------------------------------------------------------
### 1D Wavelet functions Morlet, Paul, or DOG
### Copyright (C) 1995-2021, Christopher Torrence and Gilbert P.Compo
### Python version written by Evgeniya Predybaylo (2014) and edited by Michael von Papen (2018)
### ------------------------------------------------------------------------------------
def wave_bases(mother, k, scale, param):
    n = len(k)
    kplus = np.array(k > 0.0, dtype=float)

    if mother == "MORLET":  # -----------------------------------  Morlet
        if param == -1:
            param = 6.0

        k0 = np.copy(param)
        # calc psi_0(s omega) from Table 1
        expnt = -((scale * k - k0) ** 2) / 2.0 * kplus
        norm = np.sqrt(scale * k[1]) * (np.pi ** (-0.25)) * np.sqrt(n)
        daughter = norm * np.exp(expnt)
        daughter = daughter * kplus  # Heaviside step function
        # Scale-->Fourier [Sec.3h]
        fourier_factor = (4 * np.pi) / (k0 + np.sqrt(2 + k0**2))
        coi = fourier_factor / np.sqrt(2)  # Cone-of-influence [Sec.3g]
        dofmin = 2  # Degrees of freedom
    elif mother == "PAUL":  # --------------------------------  Paul
        if param == -1:
            param = 4.0
        m = param
        # calc psi_0(s omega) from Table 1
        expnt = -scale * k * kplus
        norm_bottom = np.sqrt(m * np.prod(np.arange(1, (2 * m))))
        norm = np.sqrt(scale * k[1]) * (2**m / norm_bottom) * np.sqrt(n)
        daughter = norm * ((scale * k) ** m) * np.exp(expnt) * kplus
        fourier_factor = 4 * np.pi / (2 * m + 1)
        coi = fourier_factor * np.sqrt(2)
        dofmin = 2
    elif mother == "DOG":  # --------------------------------  DOG
        if param == -1:
            param = 2.0
        m = param
        # calc psi_0(s omega) from Table 1
        expnt = -((scale * k) ** 2) / 2.0
        norm = np.sqrt(scale * k[1] / gamma(m + 0.5)) * np.sqrt(n)
        daughter = -norm * (1j**m) * ((scale * k) ** m) * np.exp(expnt)
        fourier_factor = 2 * np.pi * np.sqrt(2.0 / (2 * m + 1))
        coi = fourier_factor / np.sqrt(2)
        dofmin = 1
    else:
        print("Mother must be one of MORLET, PAUL, DOG")

    return daughter, fourier_factor, coi, dofmin

In [ ]:
### ------------------------------------------------------------------------------------
### 1D Wavelet transform
### Copyright (C) 1995-2021, Christopher Torrence and Gilbert P.Compo
### Python version written by Evgeniya Predybaylo (2014) and edited by Michael von Papen (2018)
### ------------------------------------------------------------------------------------
def wavelet(Y, dt, pad=0, dj=-1, s0=-1, J1=-1, mother=-1, param=-1, freq=None):
    n1 = len(Y)

    if s0 == -1:
        s0 = 2 * dt
    if dj == -1:
        dj = 1.0 / 4.0
    if J1 == -1:
        J1 = np.fix((np.log(n1 * dt / s0) / np.log(2)) / dj)
    if mother == -1:
        mother = "MORLET"

    # construct time series to analyze, pad if necessary
    x = Y - np.mean(Y)
    if pad == 1:
        # power of 2 nearest to N
        base2 = np.fix(np.log(n1) / np.log(2) + 0.4999)
        nzeroes = (2 ** (base2 + 1) - n1).astype(np.int64)
        x = np.concatenate((x, np.zeros(nzeroes)))

    n = len(x)

    # construct wavenumber array used in transform [Eqn(5)]
    kplus = np.arange(1, int(n / 2) + 1)
    kplus = kplus * 2 * np.pi / (n * dt)
    kminus = np.arange(1, int((n - 1) / 2) + 1)
    kminus = np.sort((-kminus * 2 * np.pi / (n * dt)))
    k = np.concatenate(([0.0], kplus, kminus))

    # compute FFT of the (padded) time series
    f = np.fft.fft(x)  # [Eqn(3)]

    # construct SCALE array & empty PERIOD & WAVE arrays
    if mother.upper() == "MORLET":
        if param == -1:
            param = 6.0
        fourier_factor = 4 * np.pi / (param + np.sqrt(2 + param**2))
    elif mother.upper() == "PAUL":
        if param == -1:
            param = 4.0
        fourier_factor = 4 * np.pi / (2 * param + 1)
    elif mother.upper() == "DOG":
        if param == -1:
            param = 2.0
        fourier_factor = 2 * np.pi * np.sqrt(2.0 / (2 * param + 1))
    else:
        fourier_factor = np.nan

    if freq is None:
        j = np.arange(0, J1 + 1)
        scale = s0 * 2.0 ** (j * dj)
        freq = 1.0 / (fourier_factor * scale)
        period = 1.0 / freq
    else:
        scale = 1.0 / (fourier_factor * freq)
        period = 1.0 / freq
    # define the wavelet array
    wave = np.zeros(shape=(len(scale), n), dtype=complex)

    # loop through all scales and compute transform
    for a1 in range(0, len(scale)):
        daughter, fourier_factor, coi, _ = wave_bases(mother, k, scale[a1], param)
        wave[a1, :] = np.fft.ifft(f * daughter)  # wavelet transform[Eqn(4)]

    # COI [Sec.3g]
    coi = (
        coi
        * dt
        * np.concatenate(
            (
                np.insert(np.arange(int((n1 + 1) / 2) - 1), [0], [1e-5]),
                np.insert(np.flipud(np.arange(0, int(n1 / 2) - 1)), [-1], [1e-5]),
            )
        )
    )
    wave = wave[:, :n1]  # get rid of padding before returning

    return wave, period, scale, coi

****
### Parte 2: Carrega e prepara os dados
**Dados utilizados**: Dados biogeoquímicos com uma resolução temporal de 1 hora coletados pelo *Autonomous Surface Vehicle* ASV_Saildrone1020 (EXPOCODE 32DB20190119) no Oceano Austral entre 2019/01/19 e 2019/08/03. Os dados se encontram no arquivo `saildrone_data.csv` o qual foi gerado pela união de dois datasets: os [dados hidrográficos de superficie](https://data.saildrone.com/id/2091) e os [dados de pressão parcial de CO<sub>2</sub>](https://doi.org/10.25921/6zja-cg56). Para o analise só foram usados dados de temperatura superficial do mar (SST) e a diferencia de presão parcial de pCO2 entre o oceano e a atmosfera (dpco2).

In [ ]:
### ------------------------------------------------------------------------------------
### Load the saildrone time-series
### ------------------------------------------------------------------------------------
saildrone_data = pd.read_csv("./Data/saildrone_data.csv", parse_dates=[0]).set_index("time")
saildrone_data = saildrone_data.query(
    "index < 20190526"
)  # Subset the data to avoid a large data gap that start the 2019/05/26
saildrone_data

In [ ]:
### ------------------------------------------------------------------------------------
### Fill the gaps using an Piecewise Cubic Hermite Polynomial interpolation
### ------------------------------------------------------------------------------------
# This interpolation is commonly used in oceanographic studies
# (i.e. doi.org/10.1175/JTECH-D-19-0211.1) because preserves the shape of the data (is
# strictly bounded by the data points) and respects monotonicity.
saildrone_interpolated = saildrone_data.interpolate(
    method="pchip", limit_direction="forward", limit_area="inside", axis=0
).dropna(subset=["dpco2"])

In [ ]:
### ------------------------------------------------------------------------------------
### Extract the sst & dpco2 to individual variables and detrend the data
### ------------------------------------------------------------------------------------
sst_detrended = cubic_detrend(saildrone_interpolated.mean_sst)
dpco2_detrended = cubic_detrend(saildrone_interpolated.dpco2)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the clean series to check if everything is ok
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
### Subplot 1: SST
sst_detrended.plot(ax=ax1, color="red")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("Detrended SST during the Saildrone 32DB20190119", loc="left")
ax1.set_ylabel("°C")
ax1.set_xlabel(None)
### Subplot 2: ∆pCO2
dpco2_detrended.plot(ax=ax2, color="blue")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Detrended ∆pCO$_2$ during the Saildrone 32DB20190119", loc="left")
ax2.set_ylabel("µatm")
ax2.set_xlabel(None)
### Final touch
fig.tight_layout()
plt.show()

****
### Parte 3: Calcula o espectro de energia mediante o método de Welch 

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the Power Spectral Density (PSD)
### ------------------------------------------------------------------------------------
# As observations have a time resolution of 1 hour, the parameter fs = 24 returns
# frequency as cycles/day
sst_psd = signal.welch(sst_detrended, fs=24, window="hamming", nperseg=256)
dpco2_psd = signal.welch(dpco2_detrended, fs=24, window="hamming", nperseg=256)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the Power Spectral Density (PSD)
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
### Subplot 1: SST
ax1.semilogy(sst_psd[0], sst_psd[1], color="red")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("SST Power Spectral Density (PSD)", loc="left")
ax1.set_ylabel("PSD")
ax1.set_xlabel(None)
### Subplot 2: ∆pCO2
ax2.semilogy(dpco2_psd[0], dpco2_psd[1], color="blue")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("∆pCO$_2$ Power Spectral Density (PSD)", loc="left")
ax2.set_ylabel("PSD")
ax2.set_xlabel("Frequency (cycles day$^{-1}$)")
### Final touch
fig.tight_layout()
plt.show()

In [ ]:
### ------------------------------------------------------------------------------------
### Find the peaks and calculate the periods fos SST
### ------------------------------------------------------------------------------------
sst_peaks = signal.find_peaks(sst_psd[1], height=0.01)
sst_psdv = sst_peaks[1]
sst_freqs = np.array([sst_psd[0][n] for n in sst_peaks[0]])
sst_table = pd.DataFrame(
    {"Frecuency": sst_freqs, "PSD": [*sst_psdv.values()][0], "Period": 1 / sst_freqs}
).sort_values(by="PSD", ascending=False)
sst_table

In [ ]:
### ------------------------------------------------------------------------------------
### Find the peaks and calculate the periods fos ∆pCO2
### ------------------------------------------------------------------------------------
pco2_peaks = signal.find_peaks(dpco2_psd[1], height=0.5)
pco2_psdv = pco2_peaks[1]
pco2_freqs = np.array([dpco2_psd[0][n] for n in pco2_peaks[0]])
pco2_table = pd.DataFrame(
    {"Frecuency": pco2_freqs, "PSD": [*pco2_psdv.values()][0], "Period": 1 / pco2_freqs}
).sort_values(by="PSD", ascending=False)
pco2_table.head(5)

****
### Parte 4: Faz a analise de ondeletas

In [ ]:
### ------------------------------------------------------------------------------------
### Configure the main inputs for the wavelet analysis
### ------------------------------------------------------------------------------------
sampling_time = 1 / 24  # Hourly observations, day is the time unit
pad = 1  # Pad the time series with zeroes (improves fft and reduce edge effect)
dj = 0.25  # Spacing between discrete scales (periods) (0.25 = 4 sub-scales)
s0 = 2 * sampling_time  # The smallest scale of the wavelet
j1 = 9 / dj  # The number of scales (periods) minus one

In [ ]:
### ------------------------------------------------------------------------------------
### Make the wavelet transform
### ------------------------------------------------------------------------------------
# SST
sst_wave_morlet = wavelet(sst_detrended, sampling_time, pad, dj, s0, j1, "MORLET")
sst_wave_paul = wavelet(sst_detrended, sampling_time, pad, dj, s0, j1, "PAUL")
sst_wave_dog = wavelet(sst_detrended, sampling_time, pad, dj, s0, j1, "DOG")
# ∆pCO2
dpco2_wave_morlet = wavelet(dpco2_detrended, sampling_time, pad, dj, s0, j1, "MORLET")
dpco2_wave_paul = wavelet(dpco2_detrended, sampling_time, pad, dj, s0, j1, "PAUL")
dpco2_wave_dog = wavelet(dpco2_detrended, sampling_time, pad, dj, s0, j1, "DOG")

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the wavelet power spectrum
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(30, 10), dpi=300)
ax1 = fig.add_subplot(231)
ax2 = fig.add_subplot(232)
ax3 = fig.add_subplot(233)
ax4 = fig.add_subplot(234)
ax5 = fig.add_subplot(235)
ax6 = fig.add_subplot(236)
### Subplot 1: SST (Morlet)
ax1.contourf(sst_detrended.index, sst_wave_morlet[1], (np.abs(sst_wave_morlet[0])) ** 2)
ax1.fill_between(
    sst_detrended.index,
    sst_wave_morlet[3] * 0 + sst_wave_morlet[1][-1],
    sst_wave_morlet[3],
    facecolor="white",
    alpha=0.3,
)
ax1.plot(sst_detrended.index, sst_wave_morlet[3], "white")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_ylim(np.min(sst_wave_morlet[1]), np.max(sst_wave_morlet[1]))
ax1.invert_yaxis()
ax1.set_title("SST wavelet power spectrum (Morlet)", loc="left")
ax1.set_ylabel("Period (days)")
ax1.set_xlabel(None)
### Subplot 2: SST (Paul)
ax2.contourf(sst_detrended.index, sst_wave_paul[1], (np.abs(sst_wave_paul[0])) ** 2)
ax2.fill_between(
    sst_detrended.index,
    sst_wave_paul[3] * 0 + sst_wave_paul[1][-1],
    sst_wave_paul[3],
    facecolor="white",
    alpha=0.3,
)
ax2.plot(sst_detrended.index, sst_wave_paul[3], "white")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_ylim(np.min(sst_wave_paul[1]), np.max(sst_wave_paul[1]))
ax2.invert_yaxis()
ax2.set_title("SST wavelet power spectrum (Paul)", loc="left")
ax2.set_ylabel("Period (days)")
ax2.set_xlabel(None)
### Subplot 2: SST (Dog)
ax3.contourf(sst_detrended.index, sst_wave_dog[1], (np.abs(sst_wave_dog[0])) ** 2)
ax3.fill_between(
    sst_detrended.index,
    sst_wave_dog[3] * 0 + sst_wave_dog[1][-1],
    sst_wave_dog[3],
    facecolor="white",
    alpha=0.3,
)
ax3.plot(sst_detrended.index, sst_wave_dog[3], "white")
ax3.grid(linestyle="dashed", alpha=0.3)
ax3.set_ylim(np.min(sst_wave_dog[1]), np.max(sst_wave_dog[1]))
ax3.invert_yaxis()
ax3.set_title("SST wavelet power spectrum (dog)", loc="left")
ax3.set_ylabel("Period (days)")
ax3.set_xlabel(None)
### Subplot 4: dpco2 (Morlet)
ax4.contourf(
    dpco2_detrended.index, dpco2_wave_morlet[1], (np.abs(dpco2_wave_morlet[0])) ** 2
)
ax4.fill_between(
    dpco2_detrended.index,
    dpco2_wave_morlet[3] * 0 + dpco2_wave_morlet[1][-1],
    dpco2_wave_morlet[3],
    facecolor="white",
    alpha=0.3,
)
ax4.plot(dpco2_detrended.index, dpco2_wave_morlet[3], "white")
ax4.grid(linestyle="dashed", alpha=0.3)
ax4.set_ylim(np.min(dpco2_wave_morlet[1]), np.max(dpco2_wave_morlet[1]))
ax4.invert_yaxis()
ax4.set_title("dpco2 wavelet power spectrum (Morlet)", loc="left")
ax4.set_ylabel("Period (days)")
ax4.set_xlabel(None)
### Subplot 5: dpco2 (Paul)
ax5.contourf(
    dpco2_detrended.index, dpco2_wave_paul[1], (np.abs(dpco2_wave_paul[0])) ** 2
)
ax5.fill_between(
    dpco2_detrended.index,
    dpco2_wave_paul[3] * 0 + dpco2_wave_paul[1][-1],
    dpco2_wave_paul[3],
    facecolor="white",
    alpha=0.3,
)
ax5.plot(dpco2_detrended.index, dpco2_wave_paul[3], "white")
ax5.grid(linestyle="dashed", alpha=0.3)
ax5.set_ylim(np.min(dpco2_wave_paul[1]), np.max(dpco2_wave_paul[1]))
ax5.invert_yaxis()
ax5.set_title("dpco2 wavelet power spectrum (paul)", loc="left")
ax5.set_ylabel("Period (days)")
ax5.set_xlabel(None)
### Subplot 6: dpco2 (Dog)
ax6.contourf(dpco2_detrended.index, dpco2_wave_dog[1], (np.abs(dpco2_wave_dog[0])) ** 2)
ax6.fill_between(
    dpco2_detrended.index,
    dpco2_wave_dog[3] * 0 + dpco2_wave_dog[1][-1],
    dpco2_wave_dog[3],
    facecolor="white",
    alpha=0.3,
)
ax6.plot(dpco2_detrended.index, dpco2_wave_dog[3], "white")
ax6.grid(linestyle="dashed", alpha=0.3)
ax6.set_ylim(np.min(dpco2_wave_dog[1]), np.max(dpco2_wave_dog[1]))
ax6.invert_yaxis()
ax6.set_title("dpco2 wavelet power spectrum (dog)", loc="left")
ax6.set_ylabel("Period (days)")
ax6.set_xlabel(None)
### Final touch
fig.tight_layout()
plt.show()

****
### Parte 5: Discussão

Duas análises de frequência foram realizadas para os dados de SST e ΔpCO2 provenientes do Saildrone 32DB20190119 entre 2019/01/19 e 2019/05/26, obtendo assim dois produtos: um periodograma (usando o método de Welch) e um escalograma (usando três wavelets-mãe diferentes: Morlet, Paul e DOG). A seguir, são discutidos os principais resultados para cada produto.
  
Os periodogramas de SST e ΔpCO2 dos dados brutos (sem aplicar nenhum filtro) apresentaram um comportamento semelhante, marcado pela presença de um pico dominante de baixa frequência correspondente a um período de ~10 dias. Alguns picos secundários foram identificados em frequências maiores (correspondentes a períodos de ~0.7 dias), porém sua contribuição não foi significativa em comparação com o pico dominante. Se o pesquisador tivesse interesse em analisar a variabilidade de alta frequência dos dados, é aconselhável a aplicação de um filtro passa-alta para isolar apenas as frequências de interesse. A variabilidade em um período de 10 dias na região pode ter sua origem em diversos processos meteoceanográficos, como atividade de mesoescala, passagem de tempestades, entre outros.  
  
Os escalogramas, em geral, mostraram um resultado semelhante aos encontrados nos periodogramas: a variabilidade de SST e ΔpCO2 encontra-se associada à baixa frequência, com períodos dominantes na variabilidade de 10 a 40 dias, dependendo do tipo de wavelet e do período avaliado. No entanto, a utilização de diferentes wavelets ocasionou uma mudança notável na forma dos escalogramas. Por exemplo, usando a wavelet "Morlet", observa-se que a variabilidade de SST entre 2019/03/01 e 2019/04/01 está marcada por um período de 30 a 45 dias. Já para a wavelet "Paul", o pico de energia corresponde a um período de 40 a 45 dias entre 15/03/2019 e 10/04/2019. A wavelet derivada da Gaussiana (DOG) mostrou períodos semelhantes aos encontrados pela wavelet "Paul", porém apresentando mudanças na localização temporal dos diferentes componentes de frequência.